# Image Classification with CNN and Transfer Learning

#### Get the current working directory

In [1]:
import os
PATH = os.getcwd()

#### Define the data path

In [2]:
DATA_PATH = os.path.join(PATH, 'hack101_data')
data_dir_list = os.listdir(DATA_PATH)

#### Get the list of folders inside data path

In [3]:
print(data_dir_list)

['airplanes', 'BACKGROUND_Google', 'bonsai', 'brain', 'buddha', 'butterfly', 'car_side', 'chair', 'chandelier', 'cougar_face', 'crab', 'crayfish', 'dalmatian', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'euphonium', 'ewer', 'Faces', 'Faces_easy', 'ferry', 'flamingo', 'grand_piano', 'hawksbill', 'helicopter', 'ibis', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'Leopards', 'llama', 'lotus', 'menorah', 'minaret', 'Motorbikes', 'revolver', 'schooner', 'scorpion', 'soccer_ball', 'starfish', 'stop_sign', 'sunflower', 'trilobite', 'umbrella', 'watch', 'yin_yang']


#### Required variables declaration and initialization

In [4]:
img_rows=224
img_cols=224
num_channel=3

num_epoch=100
batch_size=32

img_data_list=[]
classes_names_list=[]
classes_list=[]
counter=0

#### Read the images and store them in the list

In [5]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)    
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)
        classes_list.append(counter)
    counter=counter+1

Loading images from airplanes folder

Loading images from BACKGROUND_Google folder

Loading images from bonsai folder

Loading images from brain folder

Loading images from buddha folder

Loading images from butterfly folder

Loading images from car_side folder

Loading images from chair folder

Loading images from chandelier folder

Loading images from cougar_face folder

Loading images from crab folder

Loading images from crayfish folder

Loading images from dalmatian folder

Loading images from dolphin folder

Loading images from dragonfly folder

Loading images from electric_guitar folder

Loading images from elephant folder

Loading images from euphonium folder

Loading images from ewer folder

Loading images from Faces folder

Loading images from Faces_easy folder

Loading images from ferry folder

Loading images from flamingo folder

Loading images from grand_piano folder

Loading images from hawksbill folder

Loading images from helicopter folder

Loading images from ibis fold

#### Get the number of classes

In [6]:
num_classes = len(classes_names_list)
print(num_classes)

49


####  Image preprocessiong

In [7]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [8]:
print (img_data.shape)

#img_data = img_data.reshape(img_data.shape[0], img_data.shape[1], img_data.shape[2], num_channel)

#print (img_data.shape)

(5367, 224, 224, 3)


In [9]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape
num_of_samples
input_shape

(224, 224, 3)

In [10]:
#classes = np.ones((num_of_samples,), dtype='int64')

#classes[0:110]=0
#classes[110:220]=1
classes=np.asarray(classes_list)
classes

array([ 0,  0,  0, ..., 48, 48, 48])

Convert class labels to numberic using on-hot encoding

In [11]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

Using TensorFlow backend.


In [12]:
classes.shape

(5367, 49)

In [13]:
classes[5]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

#### Shuffle the dataset

In [14]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

#### Split the dataset

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [16]:
y_test.shape

(1074, 49)

####  Defining the model

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [26]:
from keras.layers import Input, Dense
image_input = Input(shape=(img_rows, img_cols, num_channel))

###### Transfer Learning - 2

In [28]:
# Training the feature extraction also
from keras.applications.vgg16 import VGG16
model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()

553467904/553467096 [==============================] - 5883s 11us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block

In [29]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)

In [30]:
from keras.models import Model
custom_vgg_model2 = Model(image_input, out)
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [31]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

In [32]:
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [2]:
custom_vgg_model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

NameError: name 'custom_vgg_model2' is not defined

In [1]:
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data=(X_test, y_test))

NameError: name 'custom_vgg_model2' is not defined

In [80]:
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

162/162 [==============================] - 121s 744ms/step
[INFO] loss=0.2808, accuracy: 91.9753%


In [81]:
Y_test_pred = custom_vgg_model2.predict(X_test)

In [82]:
y_test_pred = np.argmax(Y_test_pred, axis=1)
print(y_test_pred)

[2 2 1 0 1 2 3 3 1 3 1 3 2 3 3 2 0 3 3 2 1 3 0 0 0 1 3 3 0 3 2 1 2 2 0 3 1
 1 3 2 1 0 1 0 1 1 1 0 2 0 2 3 1 1 2 2 0 2 3 0 3 0 0 3 1 1 1 2 3 2 0 0 2 0
 3 0 1 3 1 2 1 1 1 2 2 1 1 3 1 0 2 1 0 0 3 0 2 2 2 3 0 3 0 1 3 3 0 3 3 2 2
 3 1 0 2 1 3 2 3 3 1 0 3 0 2 2 1 2 1 0 2 3 0 1 3 0 1 2 0 3 1 0 3 3 3 0 0 3
 3 0 1 1 0 3 3 3 3 1 1 2 1 0]


In [83]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_test_pred))

[[36  5  0  0]
 [ 2 36  1  1]
 [ 1  0 32  0]
 [ 0  1  2 45]]
